This code is to process Sentinel 1 GRD with Snap to Gamma0 Terrain Flattening. The script is forked from John Truckenbrodt (S1_ARD).
To make it work one has to follow all the setup instructions of Truckenbrodts script.

In [ ]:
import re
import os
from pyroSAR.auxdata import dem_autoload, dem_create
from spatialist import gdalwarp, Vector

maindir = '/home/rus/Sentinel1/S1_ARD/data/'
snap5_gpt = '/usr/local/snap/bin/gpt'
site_file = name_shp = '/home/rus/Sentinel1/Data1404/UgandaShape4326.shp'
epsg=21036
demType = 'AW3D30'
dem_id = demType.replace(' ', '-')
dem_base = 'Uganda_dem_snap_{}'.format(dem_id)
dem_dir = os.path.join(maindir, 'DEM')
if not os.path.isdir(dem_dir):
    os.makedirs(dem_dir)
dem_snap = os.path.join(dem_dir, dem_base + '.tif')
print(dem_snap.replace(maindir, '.'))
if not os.path.isfile(dem_snap):
    with Vector(site_file) as site:
        vrt = dem_autoload(geometries=[site], 
                           demType=demType,
                           vrt='/vsimem/{}.vrt'.format(dem_base), 
                           buffer=0.1)
    # create a DEM GTiff file from the VRT
    dem_create(src=vrt, dst=dem_snap, t_srs=epsg, tr=(20, 20), geoid_convert=True)

In [ ]:
from pyroSAR import snap
from spatialist.ancillary import finder

scenes = finder(os.path.join(maindir, 's1'), ['S1*zip'])

scenedir = os.path.join(maindir, 'SNAP', 'stack')
for scene in scenes:
    print(os.path.basename(scene))
    snap.geocode(infile=scene,
                 externalDEMFile=dem_snap, shapefile=site_file,
                 externalDEMApplyEGM=False, t_srs=epsg,
                 outdir=scenedir, tr=20, scaling='db', groupsize=1,
                 export_extra=['incidenceAngleFromEllipsoid',
                               'localIncidenceAngle',
                               'projectedLocalIncidenceAngle',
                               'DEM'],
                 gpt_exceptions={'Terrain-Flattening': snap5_gpt})
    print('-' * 10)